In [64]:
from lib.v1_mysql_op import lang_lib
from lib.v1_common import mysql_op
import time
import sys, os
import datetime as dt
import pymysql as db
import pandas as pd
pd.set_option('display.max_columns', 500)

In [65]:
lang_dw = lang_lib(db_name='new_dw')
cur_lang = lang_dw.conn.cursor()

In [66]:
(dt.date.today() + dt.timedelta(days=-1)).strftime("%Y-%m-%d")

'2020-04-21'

In [67]:
def get_xday_user_watch_dur(x=0):
    end = (dt.date.today() + dt.timedelta(days=-1)).strftime("%Y-%m-%d") # 規定至少抓前一日資料
    start = (dt.date.today() + dt.timedelta(days=-1-x)).strftime("%Y-%m-%d")
    
    result_sql = f"""
    select pfid, ts
    from lang_share.duration_log_from_prod
    where length(pfid)=7 
    and reg_time between '{start} 00:00:00' and '{end} 23:59:59';
    """
    
    cur_lang.execute(result_sql)
    result = cur_lang.fetchall()
    field_list = 'pfid, watch_duration'
    
    return result, field_list

In [68]:
def get_xday_user_watch_days(x=0):
    end = (dt.date.today() + dt.timedelta(days=-1)).strftime("%Y-%m-%d") # 規定至少抓前一日資料
    start = (dt.date.today() + dt.timedelta(days=-1-x)).strftime("%Y-%m-%d")
    
    result_sql = f"""
    select pfid, count(distinct(left(reg_time, 10))) as watch_days
    from lang_share.duration_log_from_prod
    where length(pfid)=7
    and reg_time between '{start} 00:00:00' and '{end} 23:59:59'
    group by pfid;
    """
    
    cur_lang.execute(result_sql)
    result = cur_lang.fetchall()
    field_list = 'pfid, watch_days'
    
    return result, field_list

In [43]:
get_xday_user_watch_days(x=7)[0]

(('1000002', 6),
 ('1000006', 3),
 ('1000010', 8),
 ('1000011', 6),
 ('1000015', 8),
 ('1000033', 4),
 ('1000041', 5),
 ('1000053', 2),
 ('1000054', 8),
 ('1000060', 7),
 ('1000069', 5),
 ('1000072', 5),
 ('1000077', 1),
 ('1000081', 3),
 ('1000086', 3),
 ('1000089', 5),
 ('1000103', 5),
 ('1000124', 2),
 ('1000126', 1),
 ('1000139', 1),
 ('1000167', 6),
 ('1000236', 1),
 ('1000273', 2),
 ('1000337', 8),
 ('1000412', 8),
 ('1000427', 2),
 ('1000452', 8),
 ('1000484', 1),
 ('1000515', 1),
 ('1000547', 8),
 ('1000597', 6),
 ('1000635', 8),
 ('1000703', 4),
 ('1000737', 7),
 ('1000752', 5),
 ('1000757', 5),
 ('1000817', 5),
 ('1000888', 8),
 ('1000923', 2),
 ('1000928', 1),
 ('1000969', 6),
 ('1000974', 2),
 ('1000983', 7),
 ('1000999', 2),
 ('1001069', 1),
 ('1001116', 8),
 ('1001123', 7),
 ('1001125', 7),
 ('1001127', 8),
 ('1001131', 8),
 ('1001139', 2),
 ('1001149', 1),
 ('1001156', 2),
 ('1001190', 1),
 ('1001303', 5),
 ('1001311', 8),
 ('1001313', 6),
 ('1001335', 2),
 ('1001342', 1

In [56]:
# 先把一年的 live_id 都抓出來看會跑多久: 10 mins
select_liveid_sql = f"""
select live_id
from lang_share.duration_log_from_prod 
where reg_time between '2019-04-20 00:00:00' and '2020-04-20 23:59:59';
"""
s = time.time()
cur_lang.execute(select_liveid_sql)
liveid_lst = cur_lang.fetchall()
e = time.time()
print(e-s)

525.7969379425049


In [69]:
# 先把一年的 live_id, 觀看時數抓出看會跑多久:
select_liveid_sql = f"""
select left(live_id,7) as live_id, ts
from lang_share.duration_log_from_prod force index(idx_date)
where reg_time between '2020-04-21 00:00:00' and '2020-04-21 23:59:59'
#and left(live_id,7)='2132991'
and ts>60
and left(live_id,7)!=pfid;

"""
s = time.time()
cur_lang.execute(select_liveid_sql)
liveid_lst = cur_lang.fetchall()
e = time.time()
print(e-s)

26.805288076400757


2020-04-22 11:00:40,710| ERROR   | Socket exception: Operation timed out (60)
2020-04-22 14:05:14,789| ERROR   | Socket exception: Connection reset by peer (54)


In [61]:
liveid_lst[0][0]

'1010909Y37956iWfV'

In [63]:
tmp = ()
for i in range(len(liveid_lst)):
    tmp.append(liveid_lst[i][0])

tmp

AttributeError: 'tuple' object has no attribute 'append'